<a href="https://colab.research.google.com/github/merazlab/cuda/blob/main/gpu_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem
```
Write a program to add two array using cuda core ?
```

In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1eenvcdy
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1eenvcdy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=b10905263358bea05683b0b6e4255988fe60dabb10568338e217f12d0c0b4f2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-gpwqwdpq/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [10]:
%%cu
#include<stdio.h>
#define BS 8
#define N 10

void print(int *A,int n){
    for(int i=0; i<n; i++)
        printf("%d ",A[i]);
    printf("\n");
}

__global__ void add_array(int *A, int *B, int n){
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if(i < n) A[i] = A[i] + B[i];
}

int main(void){
    int threadsPerBlock, blocksPerGrid, n, *A, *B, *dA, *dB;
    n=N; threadsPerBlock=BS;
    blocksPerGrid = (n+BS-1)/BS;
    
    A = (int*) malloc(n*sizeof(int));
    B = (int*) malloc(n*sizeof(int));

    for(int i=0; i<n; i++) A[i]=i;
    for(int i=0; i<n; i++) B[i]=i*2;
    print(A, n);
    print(B, n);    

    cudaMalloc((void**)&dA, n*sizeof(int));
    cudaMalloc((void**)&dB, n*sizeof(int));

    cudaMemcpy(dA, A, n*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dB, B, n*sizeof(int), cudaMemcpyHostToDevice);

    add_array<<<blocksPerGrid,threadsPerBlock>>>(dA, dB, n);

    cudaMemcpy(A , dA, n*sizeof(int), cudaMemcpyDeviceToHost);

    print(A, n);
 
    cudaFree(dA);
    free(A);

    return 0;
}

0 1 2 3 4 5 6 7 8 9 
0 2 4 6 8 10 12 14 16 18 
0 3 6 9 12 15 18 21 24 27 

